In [69]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_curve



# Read Data

In [70]:
#Import Censored Data
censored_df=pd.read_csv('censored_tweets.csv')
censored_df = pd.DataFrame(censored_df)
censored_df.insert(loc=3, column='y', value=1)
censored_df = censored_df.drop(['id','lang'],axis=1)

#import regular tweets
uncensored_df=pd.read_csv('regular_tweets.csv')
uncensored_df = pd.DataFrame(uncensored_df)
uncensored_df.insert(loc=3, column='y', value=0)
uncensored_df = uncensored_df.drop(['id','lang'],axis=1)

#process tweets
uncensored_df["text"].apply(lambda t: t.lower().translate(str.maketrans('', '', string.punctuation)))
censored_df["text"].apply(lambda t: t.lower().translate(str.maketrans('', '', string.punctuation)))

#merge and shufflem
df = pd.concat([censored_df, uncensored_df], ignore_index=True, sort=False)
df = shuffle(df)
df_x=df["text"]
df_y=df["y"]

In [71]:
df

,text,y
2054,Richard Spencer—the Nazi sympathizer who blitz...,0
2989,👀 Coming soon... @StoolGametime @KFCBarstool #...,0
1624,Russia Watches Europe Commit Suicide By Migrat...,1
3726,Kanye West’s support of the President is a per...,0
3057,"In this month of May of Annointing, get prepar...",0
...,...,...
1117,1/2Leaked document from Assad regime forces in...,1
3211,Papa @Gurmeetramrahim G Bless me to do it wit...,0
2799,preview of 👋 #ทีมพีรญา #SingtoKrist #peraya...,0
1303,A large number of regime soldiers were capture...,1


In [72]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=4)
print("fraction of censored tweets", y_train.mean())

fraction of censored tweets 0.4797427652733119


In [73]:
x_train

1849    "Future leader of South Africa", when publicly...
3229                 @sujeongtheryu Lol i don't know too😂
1120    SPIELEN  #Casino - Up to GBP1000 Welcome bonus...
139     @GatestoneInst 30 day FB ban for posting pics ...
323     Joel Payne: "If we can't trust our president t...
                              ...                        
1967           @MichiChar27 !!!!!! Heck yeah buddy!!!! ♡♡
915     Dear Media...  Even if you don't like Trump, i...
1561    Reports continue to come indicating #Iran mull...
96      Today I Learned Buildings Can Get Cancer https...
3447    Share this to weed out all the immature people...
Name: text, Length: 3110, dtype: object

# Create Spam Bag of Words and Non-Spam Bag of Words

In [74]:

#get bag of censored words & bag of uncensored words
x_train_censored = x_train[y_train == 1]
x_train_uncensored = x_train[y_train == 0]

train_censored_bag = ' '.join(x_train_censored).split(' ')
train_uncensored_bag = ' '.join(x_train_uncensored).split(' ')
total_words = set(train_censored_bag).union(set(train_uncensored_bag))

In [75]:
train_censored_bow = dict()
for w in total_words:
    train_censored_bow[w] = train_censored_bag.count(w) / len(train_censored_bag)

In [76]:
train_uncensored_bow = dict()
for w in total_words:
    train_uncensored_bow[w] = train_uncensored_bag.count(w) / len(train_uncensored_bag)

# Predict on Test Set

In [77]:
def predict_text(t, verbose=False):
    #if some word doesnt appear in either censored or non-censored BOW, disregard it
    valid_words = [w for w in t if (w in train_uncensored_bag) or (w in train_censored_bag)]
    
    #get the probabilities of each valid word showing up in spam and non-spam BOW
    censored_probs = [train_censored_bow[w]+ 0.000001 for w in valid_words]
    uncensored_probs = [train_uncensored_bow[w]+ 0.000001 for w in valid_words]
    
    #print probs if requested
    if verbose:
        data_df = pd.DataFrame()
        data_df['word'] = valid_words
        data_df['censored_prob'] = censored_probs
        data_df['uncensored_prob'] = uncensored_probs
        data_df['ratio'] = [c/u if u > 0 else np.inf for c,u in zip(censored_probs, uncensored_probs)]
        print(data_df)
     
    #calculate spam score as sum of logs for all probabilities
    censored_score = sum([np.log(p) for p in censored_probs]) + np.log(y_train.mean())
    
    #calculate non-spam score as sum of logs for all probabilities
    uncensored_score = sum([np.log(p) for p in uncensored_probs]) + np.log(1-y_train.mean())
    
    #if verbose, report the two scores
    if verbose:
        print('censored score: %s'%censored_score)
        print('uncensored score: %s'%uncensored_score)
   
    #if spam score is higher, mark this as spam
    return (censored_score >= uncensored_score)

In [83]:
predict_text('Hitler was the worst leader'.split(), verbose=True)

     word  censored_prob  uncensored_prob       ratio
0  Hitler       0.000294         0.000001  293.615450
1     was       0.003034         0.003287    0.922954
2     the       0.030619         0.025169    1.216555
3   worst       0.000107         0.000106    1.014545
4  leader       0.000267         0.000036    7.426287
censored score: -35.51899809077263
uncensored score: -43.25550961655299


True

In [79]:
predictions = x_test.apply(lambda t: predict_text(t.split()))

In [80]:
confusion_matrix(y_test, predictions)
print(classification_report(y_test, predictions))
print("Accuracy: ", mnb.score(x_testcv, y_test))

              precision    recall  f1-score   support

           0       0.77      0.76      0.76       381
           1       0.77      0.78      0.77       397

    accuracy                           0.77       778
   macro avg       0.77      0.77      0.77       778
weighted avg       0.77      0.77      0.77       778



NameError: name 'mnb' is not defined

In [ ]:
scores = mnb.predict_proba(x_testcv)
fpr, tpr, _ = roc_curve(y_test, scores[:, 1])
plt.plot(fpr, tpr)

#Plot
plt.title('ROC Curves')
plt.xlabel('False positive rate'); plt.ylabel('True positive rate')
plt.legend(['Logistic Regression'])
plt.show()

Fraction uncensored identified as censored: 0.7956730769230769
